In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import math
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from selenium.webdriver.common.by import By
import json
import pygsheets
from google.oauth2 import service_account
import platform
import datetime
from dateutil.relativedelta import relativedelta
import sys
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

class MyChromeDriver(webdriver.Chrome):

    def __init__(self, headless=False, maximize=True):

        options = webdriver.ChromeOptions()
        if headless:
            options.add_argument('--headless')

        webdriver.Chrome.__init__(self, service=ChromeService(ChromeDriverManager().install()), options=options)

        if maximize:
            self.maximize_window()

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

if driver is None:
        my_driver = MyChromeDriver()
        time.sleep(10)
else:
    my_driver = driver

os.getcwd()

'c:\\Users\\SANDBOX\\Desktop\\Sandbox\\crawling'

In [2]:
# # 크롬 드라이버 옵션 설정 : 지정하면 크롬 아예 안보임
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
#driver = webdriver.Chrome() # 크롬 옵션 지정 시 options=chrome_options 추가

# 올해 광고 영상 크롤링
title_list = []
video_list = []
channel_list = []
keyword_list = []
include_list = ['공부', '여행', '먹방', '메이크업', '캠핑', '다이어트', '리뷰', '브이로그', '운동', '추천', '코디']

for k in range(len(include_list)) :
    try :
        search_keyword = '%23'+include_list[k]+'%2B유료광고포함'
        url = 'https://www.youtube.com/results?search_query='+search_keyword
        my_driver.get(url)
        time.sleep(2)

        my_driver.find_element(By.CSS_SELECTOR,
                               '#filter-button > ytd-button-renderer > yt-button-shape > button > yt-touch-feedback-shape > div > div.yt-spec-touch-feedback-shape__fill').click()
        time.sleep(2)
        
        my_driver.find_element(By.XPATH,
                               '/html/body/ytd-app/ytd-popup-container/tp-yt-paper-dialog/ytd-search-filter-options-dialog-renderer/div[2]/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[5]/a').click()
        time.sleep(2)
    
        # try :
        #     upload_date.find_elements(By.ID,'endpoint')[3].click()
        #     time.sleep(2)
        # except :
        #     my_driver.quit()
        #     continue

        last_height = my_driver.execute_script("return document.documentElement.scrollHeight")

        while True:
            my_driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(1)

            new_height = my_driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                break

            last_height = new_height

        pageString = my_driver.page_source
        soup = BeautifulSoup(pageString, features='html.parser')

        for i in tqdm(range(len(soup.find_all('ytd-video-renderer',{'class':'style-scope ytd-item-section-renderer'})))): 
        #soup.find_all('ytd-video-meta-block', 'style-scope ytd-video-renderer byline-separated'))):

            title = soup.find_all('a',{'id':'video-title'})[i].text.split('\n')[2]
            title_list.append(title)

            if 'shorts' in soup.find_all('a',{'id':'video-title'})[i].get('href'):
                video = 'https://youtu.be/'+ soup.find_all('a',{'id':'video-title'})[i].get('href').split('/')[2]
                video_list.append(video)

            else :
                video = 'https://youtu.be/'+ soup.find_all('a',{'id':'video-title'})[i].get('href').split('=')[1]
                video_list.append(video)

            channel = soup.find_all('yt-formatted-string',{'class':'style-scope ytd-channel-name'})[i].text
            channel_list.append(channel)

            keyword = include_list[k]
            keyword_list.append(keyword)

            print(i, title, video, channel, keyword)
            
    except :
        print('오류')
        
driver.quit()

 40%|████      | 8/20 [00:00<00:00, 32.12it/s]

0 미리 정해두는 5월 필수 공부 (책 증정이벤트 & 유료광고 포함) (중고등 과목별 교재 & 공부법 추천)  | 대치동장원장 시즌6 #자기주도학습 #스터디PT #스터디피티 https://youtu.be/XlP8zzp0VH4&pp 대치동장원장 공부
1 [유료광고포함] 새벽 4시 기상해서 미국 하이틴으로 변신하기! 미드 유포리아 캐씨 모닝루틴 챌린지 💖 https://youtu.be/ejj-BAA7lwo&pp 해쭈 [HAEJOO] 공부
2 [공시생 브이로그2 #8] 경각심 좀 가져야 할 싹퉁바가지 컴백ㅣ살다 살다 별 초파리를 다 본다ㅣ운동 좀 하려다 입맛만 돌아벌인 나ㅣ저는 옷 안사요 책 사요,,,📚 https://youtu.be/Wr-mT74Gn2A&pp 뎌니Dyeoni 공부
3 직장인 삼킨 최우식 & 멈추지 않는 알람⏰ 소리 #highlight #유료광고포함 #서진이네 EP.8 https://youtu.be/L8mMwsnZwkU&pp tvN 공부
4 ENG) vlog 가을맞이 자취방 꾸미고 매운어묵김밥과 신라면 더레드 먹으며 매운 요리 잔뜩 만드는 자취생 일상ㅣ낙지볶음, 지코바치킨, 웨지감자ㅣ침구교체, 토익공부, 성수동 핫플 https://youtu.be/uBmj8nymexc&pp Orosina 오로시나 공부
5 하필 고3때 외계인 쳐들어와ㅠ 전교생이 "군필 고딩"이 됨ㅋ 열받은 배그의 민족 방과후 육군훈련으로 점점 특수부대급🔥으로 성장하는 개쩌는 설정의 드라마 나왔다ㅋㅋ https://youtu.be/_bEK-vqluj0&pp 고몽 공부
6 𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭 부장님도 같이 리듬 타게 되는 직장인 노동요 https://youtu.be/qabCoid-K-8&pp 때껄룩ᴛᴀᴋᴇ ᴀ ʟᴏᴏᴋ 공부
7 문해력 이슈 | 문제는 영상 때문이다🤔? #highlight #유료광고포함#유퀴즈온더블럭 | YOU QUIZ ON THE BLOCK EP.204 https://youtu.be/iiXDssiA_Ts&pp 유 퀴즈 온 더 튜브 공부
8 경제금융학과 모범생들의 하루 http

 80%|████████  | 16/20 [00:00<00:00, 33.06it/s]

10 SUB 사이다) 지방출신은 꼴통 집합소? 빡대가리라고 비하하다 참교육 당한 전교1등ㅋㅋ | [루대숲] - 105번째 사연 https://youtu.be/J-jwQKYGVBo&pp 루디고GO 공부
11 eng) 카이스트 개강 브이로그 | 과제&공부 꿀팁🍯, 도서관 공부, 개강파티, 운동 | 미러 사용법 | KAIST study vlog https://youtu.be/ckAbdCa8Gk8&pp 밍주라네 Mingjurane 공부
12 경제공부는 이게 정석이지 https://youtu.be/Dhkx7rudBy4 윤루카스 공부
13 𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭 나한테 전생이 있었던가 사극 과몰입 플리 https://youtu.be/AUOeYg43yTs&pp 때껄룩ᴛᴀᴋᴇ ᴀ ʟᴏᴏᴋ 공부
14 맨해튼 프로젝트 1등공신! 인간의 두뇌를 초월한 슈퍼천재 폰 노이만 https://youtu.be/34sEM5J9sHo&pp 리뷰엉이: Owl's Review 공부
15 SNS 팔로워수로 권력이 나뉘는 인플루언서맘들의 잘난척과 허세 뒤 숨겨진 갑질러의 모습…! 펜트하우스급 초고급 아파트 쎈케 엄마들의 소름끼치는 저격과 나락《행복배틀》#ena채널 https://youtu.be/uhPy9uyolGw&pp 고몽 공부
16 수능 가산점 때문에 군대에 간 고등학생들에게 벌어진 아주 아주 최악의 사건..! [방과 후 전쟁활동] 4,5,6회 https://youtu.be/wKtDGxQFajk&pp 고몽 공부
17 홈쇼핑 수도승의 풀소유 인생💰 (홈쇼핑 말고 다른 유료광고 포함)│Ep.79 https://youtu.be/Nfnk_AeTVk0&pp 구라철 공부
18 챗GPT를 신빡하게 사용하는 방법  ➡️ GPT4 무료사용법 포함 https://youtu.be/Q0HKGCUH1DA&pp 무빙워터 공부


100%|██████████| 20/20 [00:00<00:00, 33.36it/s]


...

8 발목뚠뚠❌ 얇고쫀득해 다리가예뻐보이는 만능코디템 🧦기본&니트양말🧦 ㅣ 전직 다리모델이 만든 국내생산 브랜드양말 론칭특가! (단 일주일간 3장 9,900원💸) https://youtu.be/IwUaaEd_eYA&pp 뷰티트레이너 쭈언니 코디
9 봄 소풍 나들이 완벽한 인생샷을 위한 코디 l 활동성🙅🏻‍♀️ THE PERFECT PICNIC OUTFITS #picnic#cody#goodvibes#스타일링 https://youtu.be/lMx2y_tRZSc STYLE BOOK9 코디
10 인⭐그램 업로드에 빠진 이효리 남다른(?) 출근길 스타일링 #유료광고포함 #댄스가수유랑단 EP.4 | tvN 230615 방송 https://youtu.be/3BA3k748akY&pp tvN 코디
11 세상에서 가장 '미안한' 직업? 장기구득코디네이터｜크랩 (유료 광고 포함) https://youtu.be/l3r6TghsgYo&pp 크랩 KLAB 코디
12 요즘 뭐 입지?🌸 일교차 심한 환절기 코디🌷 Get dressed with me! https://youtu.be/oFk3vxctIMw 한별Hanbyul 코디
13 인스타 반응이 너무 폭발적이어서 안 올릴 수가 없었던 이 날 코디🫢 #윗유하우스 https://youtu.be/-G5ieAhGAqY 나나씨 Nanasi 코디


 90%|█████████ | 18/20 [00:00<00:00, 26.57it/s]

14 봄 패션 코디 준비해야죠? LMC 23SS 신상 쇼핑 둘러보기..! 냠냠 https://youtu.be/4YETlJhbEII&pp 스타일가이드 최겨울 코디
15 애매한 날씨에 뭐 입지?💧 일교차 심한 날 봄옷 코디! #shorts https://youtu.be/eQN5EreRC9I 한별Hanbyul 코디
16 무조건 개이득인 가성비 끝판왕 꿀템 4가지🔥 https://youtu.be/ucOO0qsFS5A&pp 핏더사이즈 코디
17 대학생 필수 스니커즈 추천 BEST 3 유료광고 포함 https://youtu.be/LWUIMv1L6mI 추천모아 코디
18 OOTD VLOG! 빈티지 쇼핑하고 식물도 사고 비건 지향 식사와 운동을 하였습니다.. 그리하여 나는 건강걸로 다시 태어남 (유료광고) https://youtu.be/PbnY8ueDG5Q&pp Susan, 코디


100%|██████████| 20/20 [00:00<00:00, 26.92it/s]


19 가을에 옷 잘 입으려면 이것만 기억하세요! 평범한 옷을 한 끗 차이로 고급지게 코디하는 법✔️ https://youtu.be/8Q0EDXsMC3w&pp AliceFunk 앨리스펑크 코디


In [3]:
# 데이터프레임으로 저장
data = {
    '제목': title_list,
    '비디오 URL': video_list,
    '채널': channel_list,
    '키워드': keyword_list,
}

crawling_1 = pd.DataFrame(data)
crawling_1

,제목,비디오 URL,채널,키워드
0,미리 정해두는 5월 필수 공부 (책 증정이벤트 & 유료광고 포함) (중고등 과목별 ...,https://youtu.be/XlP8zzp0VH4&pp,대치동장원장,공부
1,[유료광고포함] 새벽 4시 기상해서 미국 하이틴으로 변신하기! 미드 유포리아 캐씨 ...,https://youtu.be/ejj-BAA7lwo&pp,해쭈 [HAEJOO],공부
2,[공시생 브이로그2 #8] 경각심 좀 가져야 할 싹퉁바가지 컴백ㅣ살다 살다 별 초파...,https://youtu.be/Wr-mT74Gn2A&pp,뎌니Dyeoni,공부
3,직장인 삼킨 최우식 & 멈추지 않는 알람⏰ 소리 #highlight #유료광고포함 ...,https://youtu.be/L8mMwsnZwkU&pp,tvN,공부
4,ENG) vlog 가을맞이 자취방 꾸미고 매운어묵김밥과 신라면 더레드 먹으며 매운 ...,https://youtu.be/uBmj8nymexc&pp,Orosina 오로시나,공부
...,...,...,...,...
655,애매한 날씨에 뭐 입지?💧 일교차 심한 날 봄옷 코디! #shorts,https://youtu.be/eQN5EreRC9I,한별Hanbyul,코디
656,무조건 개이득인 가성비 끝판왕 꿀템 4가지🔥,https://youtu.be/ucOO0qsFS5A&pp,핏더사이즈,코디
657,대학생 필수 스니커즈 추천 BEST 3 유료광고 포함,https://youtu.be/LWUIMv1L6mI,추천모아,코디
658,OOTD VLOG! 빈티지 쇼핑하고 식물도 사고 비건 지향 식사와 운동을 하였습니다...,https://youtu.be/PbnY8ueDG5Q&pp,"Susan,",코디


In [22]:
# 영상 유료광고 배너 유무 & 더보기란 크롤링

driver = webdriver.Chrome()

video_url_list = []
keyword_list_2 = []
title_list = []
channel_list = []
channel_id_list = []
date_list = []
view_list = []
sub_list = []
ad_banner_list = []
description_list = []

for i in tqdm(range(len(video_list))):
    driver.get(video_list[i])
    time.sleep(3)
    
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # 영상 URL
    video_url = video_list[i]
    video_url_list.append(video_url)
    
    # 키워드
    keyword_2 = keyword_list[i]
    keyword_list_2.append(keyword_2)
    
    # 제목
    try:
        title = soup.find_all('yt-formatted-string', attrs={'class': 'ytd-watch-metadata'})[0].text
    except:
        title = '제목 크롤링 오류'
        continue
    title_list.append(title)
    
    # 채널명
    try:
        channel = soup.find_all('yt-formatted-string', {'id': 'text'})[0].text
    except:
        channel = '채널명 크롤링 오류'
        continue
    channel_list.append(channel)
    
    # 채널ID
    try:
        channel_id = str(soup).split(',"channelId":"')[1].split('","')[0]
    except:
        channel_id = '채널ID 크롤링 오류'
        continue
    channel_id_list.append(channel_id)
    
    # 업로드 일자
    try:
        date = soup.find('tp-yt-paper-tooltip', class_='style-scope ytd-watch-metadata').text.split('조회수')[1].strip().split('•')[1].strip()
    except:
        date = '업로드 일자 크롤링 오류'
        continue
    date_list.append(date)
    
    # 조회수
    try:
        view = soup.find('tp-yt-paper-tooltip', class_='style-scope ytd-watch-metadata').text.split('조회수')[1].strip().split('회')[0].strip()
    except:
        view = '조회수 크롤링 오류'
        continue
    view_list.append(view)
    
    # 구독자 수
    try:
        sub = soup.find('yt-formatted-string', {'id': 'owner-sub-count'}).text.split('구독자')[1].strip().split('명')[0].strip()
    except:
        sub = '구독자 수 크롤링 오류'
        continue
    sub_list.append(sub)
    
    # 유료 광고 영상 배너
    try:
        ad_banner = soup.find_all('div', class_='ytp-paid-content-overlay-text')[0].text
    except:
        ad_banner = 'X'
        continue
    ad_banner_list.append(ad_banner)
    
    # 더보기란
    try:
        description = soup.find_all('span', {'class': 'yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap'})[2].text
    except:
        description = '더보기 크롤링 오류'
        continue
    description_list.append(description)
    
    print(i, video_url, keyword_2, title, channel, channel_id, date, view, sub, ad_banner, description)

driver.quit()

  0%|          | 1/660 [00:08<1:30:58,  8.28s/it]

0 https://youtu.be/XlP8zzp0VH4&pp 공부 미리 정해두는 5월 필수 공부 (책 증정이벤트 & 유료광고 포함) (중고등 과목별 교재 & 공부법 추천)  | 대치동장원장 시즌6 #자기주도학습 #스터디PT #스터디피티 대치동장원장 UCiNbRpsxCHAWNvqwJ5VEuGg 2023. 4. 8. 7,387 3.35만 유료 광고 포함 대치동장원장의 자기주도학습 노하우를 체계적으로 마스터할 수 있는
CLASSU 자기주도학습 마스터 클래스
신학기 10% 할인 이벤트 중!
다음의 링크로 들어가셔서 결제하시면 10% 할인이 자동 적용 됩니다.
https://me2.do/5qgfKqju
가장 진보한 과학적인 자기주도학습 공부법 배우고 싶으면 꼭 신청하세요.

스마트맘과 혼공러를 위해 대치동장원장이 준비했습니다!
살아 있는 대치동 상위 1% 교육 노하우를
자녀의 상황에 맞춰
실천 가능한 자기주도학습로 적용할 수 있게 도와드립니다.

목차
00:00 Intro



#자기주도학습 #스터디피티 #스터디PT #대치동공부법 #실천가능공부법 #혼공공부법 #자녀교육 #자녀학습지도 #대치동자기주도학습 #학원 선택


  0%|          | 2/660 [00:14<1:17:02,  7.03s/it]

1 https://youtu.be/ejj-BAA7lwo&pp 공부 [유료광고포함] 새벽 4시 기상해서 미국 하이틴으로 변신하기! 미드 유포리아 캐씨 모닝루틴 챌린지 💖 해쭈 [HAEJOO] UC6KwCU8Y8Uw4h_Q0ptLZkqw 2023. 2. 10. 1,116,226 75.1만  💖디오디너리X해쭈 이벤트💖

📣앙큼불여우들 주-목!!!
영상 시청 후 댓글과 함께 아래 링크에 
들어가셔서 폼을 제출하시고 
디오디너리 데일리 세트 받아 가세요!!!

링크👉🏽https://forms.gle/RXgcNAFXfqrj2fcq6

참여 기간 : 2월 10~20일
당첨자 개별 연락 : 2월 24일

[제품 구매처]

💄시코르
https://chicor.com/exhibition/3337?ut...

🧴디오디너리 공식 브랜드 스토어
https://brand.naver.com/theordinary/s...

🛍️오프라인 매장

디오디너리 연남 매장: 서울시 마포구 동교동 153-1 1층
디오디너리 가로수길 매장: 서울시 강남구 신사동 533-14 1층

-

비지니스 문의는 이쪽으로 👉🏽 haejoo@sandboxnetwork.net
감성가득 인스타그램 👉🏽  https://instagram.com/hae_jooooo_
미지의 그곳 해쭈 팬카페 👉🏽  https://m.cafe.naver.com/australiahaeju


  0%|          | 3/660 [00:19<1:06:35,  6.08s/it]

...

657 https://youtu.be/LWUIMv1L6mI 코디 대학생 필수 스니커즈 추천 BEST 3 유료광고 포함 추천모아 UCMBFotsUgoPKkzh4Ek5aESA 2023. 7. 23. 250 12  #대학생패션 #대학생코디 #신발추천 #신발 #스니커즈추천 #스니커즈 #반스 #컨버스 #뉴발란스 #나이키 #아디다스 #ootd #브이로그 #패션브이로그 #아웃핏 #룩북 
휴학 2년차 화석의 캠퍼스 라이프를 회상하며 추천함미다..!


100%|█████████▉| 659/660 [59:23<00:05,  5.26s/it]

658 https://youtu.be/PbnY8ueDG5Q&pp 코디 OOTD VLOG! 빈티지 쇼핑하고 식물도 사고 비건 지향 식사와 운동을 하였습니다.. 그리하여 나는 건강걸로 다시 태어남 (유료광고) Susan, UC9edwufjEirbl9-2vns43VQ 2022. 9. 26. 63,467 20.9만 유료 광고 포함 #유료광고 #아워비건 #비건지향 #ootd 
* 본 영상은 마녀공장(아워 비건)의 유료광고를 포함하고 있습니다 *
[ OOTD(Our Vegan Of The Day) ]

저는 정말 간만에 비건 지향적인 일상을 영위하였고여,, OOTD도 쪼끔씩 기록하고, 빈티지 쇼핑도 잔뜩 했네요! 명확한 주제를 정해놓고 일주일을 보냈는데, 너무 즐겁고 리프레쉬 할 수 있는 시간이었어요. 이번 영상을 보고 혹여나 관심이 생기신 분들은 하루 한끼 정도 비건지향 식사를 해보셔요. 비건 식당 찾는 것도 재밌고, 채소와 버섯같은 재료의 맛을 온전히 느끼는 과정도 즐겁더라고요 후후.. 하루 한 끼 정도니까 부담도 없고요!!!! 하하하하

일상에 작지만 새로운 변화를 계속해서 주는 것은 삶에 대한 태도를 바꿔주는 것 같아요. 오늘도 영상 시청해주셔서 정말 감사합니다, 늘 힘이 되어요!

영상 내에서 사용한 아워 비건 제품은 아래에서 확인 하실 수 있습니다!  

- 아워 비건 어성초 시카 토너 400ml : https://bit.ly/3DK726r  
- 아워 비건 어성초 98 시카 세럼 100ml : https://bit.ly/3LzpsJf
- 아워 비건 어성초 시카 크림 100ml : https://bit.ly/3SqolgZ

~OOTD 착용 제품과 타임라인~

00:13~

포르테나 리센트 크롭 가디건
보세 나시
바이너리원 팬츠

00:17~

프라이빗룩북 t
애프터먼데이 플레어 워싱 진 3 (s)

1:23~

웅주 티셔츠.. ㅎ 몇년 전에 탑텐에서 샀대요.
파자마 팬츠 구매처를 찾을 수가 없네요. ㅠㅠ

2:38~
몇년 전 알리익스프레스에서 구매한 자켓..


100%|██████████| 660/660 [59:29<00:00,  5.41s/it]

659 https://youtu.be/8Q0EDXsMC3w&pp 코디 가을에 옷 잘 입으려면 이것만 기억하세요! 평범한 옷을 한 끗 차이로 고급지게 코디하는 법✔️ AliceFunk 앨리스펑크 UCcRTcy_GuofiJLPLMlVcGxw 2022. 9. 26. 181,983 67만 유료 광고 포함 가을 여신이 되고 싶나요..?
[본 영상은 EnC의 유료광고가 포함되어 있습니다]


여러분 안녕하세요 앨리스펑크예요!
오늘은 옷장에 무조건 갖춰야 할 가을 기본템들로
“직장인 가을 데일리 출근룩”을 준비했어요!!
분위기 있는 가을 여신이 되고 싶다면 오늘 소개해드린
다섯 가지 룩들 절대 놓치지 마세요!!!
띠릿!!!!!!🤟

💝EnC가 준비한 이벤트💝

💙 펑순이 자사몰 특별 할인 💙
- 자사몰 기획전 링크 : https://bit.ly/3MciBFG
- 카플친 전용 5,000원 쿠폰 : https://pf.kakao.com/_jmzqK
- 앨펑 PICK 아우터 최대 46% 할인 + 추가 10%
- 이너 제품 최대 60% 할인
- 기획전 제품 모두 무료배송
- 자사몰 신규회원 : 적립금 : 3,000원 + 10%쿠폰 + 무료배송쿠폰

💙 EnC 💙
자사몰 : https://bit.ly/3SWerEo
공식 인스타그램 : https://www.instagram.com/enc__official/


🎁 댓글 이벤트 – 본 이벤트는 종료되었습니다.
소개한 룩 중, 마음에 드는 룩과 그 이유를 댓글로 남기면 추첨을 통해 5명을 추첨하여 선물 증정!
증정 제품 : 라운드넥 V짜임 니트 풀오버 ENKT21203N / 색상 랜덤 증정
이벤트 기간 : 9월 26일(월) ~ 10월 5일(수)
당첨자 발표 : 10월 7일(금) 커뮤니티 발표

👚 앨리스펑크 착장 정보
아우터 및 이너제품 전부 55사이즈 착용

📍LOOK 1
아우터 : 캐시미어 블랜드 숏 핸드메이드 코트 ENJH24T91N 
https://bit.ly/3ykM7DK
상의 : 허리셔링 골지 니트 ENLW13891N 
ht

In [7]:
# # 제대로 크롤링 되는지 확인하기 위한 코드
# soup

# # 업로드 일자 확인
# soup.find('tp-yt-paper-tooltip', class_='style-scope ytd-watch-metadata').text.split('조회수')[1].strip().split('•')[1].strip()

# # 조회수 확인
# soup.find('tp-yt-paper-tooltip', class_='style-scope ytd-watch-metadata').text.split('조회수')[1].strip().split('회')[0].strip()

# # 구독자 수 확인
# soup.find_all('yt-formatted-string', {'id' : 'owner-sub-count'}).text.split('구독자')[1].strip().split('명')[0].strip()

# # 채널명 확인
# soup.find_all('yt-formatted-string',{'id':'text'})[0].text

# # 제목 확인
# soup.find_all('yt-formatted-string',attrs={'class' : 'ytd-watch-metadata'})[0].text

# # 유료 광고 배너 확인 (없으면 오류 뜸)
# soup.find_all('div', class_='ytp-paid-content-overlay-text')[0].text

# # 더보기란 상세내용 확인
# soup.find_all('span',{'class' : 'yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap'})[2].text

In [23]:
data_2 = {'비디오 URL': video_url_list,
          '키워드': keyword_list_2,
          '제목': title_list,
          '채널': channel_list,
          '채널ID': channel_id_list,
          '업로드 일자': date_list,
          '조회수': view_list,
          '구독자 수': sub_list,
          '유료광고 배너': ad_banner_list,
          '더보기란': description_list,
}

crawling_2 = pd.DataFrame(data_2)
crawling_2

,비디오 URL,키워드,제목,채널,채널ID,업로드 일자,조회수,구독자 수,유료광고 배너,더보기란
0,https://youtu.be/XlP8zzp0VH4&pp,공부,미리 정해두는 5월 필수 공부 (책 증정이벤트 & 유료광고 포함) (중고등 과목별 ...,대치동장원장,UCiNbRpsxCHAWNvqwJ5VEuGg,2023. 4. 8.,"7,387",3.35만,유료 광고 포함,대치동장원장의 자기주도학습 노하우를 체계적으로 마스터할 수 있는\nCLASSU 자기...
1,https://youtu.be/ejj-BAA7lwo&pp,공부,[유료광고포함] 새벽 4시 기상해서 미국 하이틴으로 변신하기! 미드 유포리아 캐씨 ...,해쭈 [HAEJOO],UC6KwCU8Y8Uw4h_Q0ptLZkqw,2023. 2. 10.,"1,116,226",75.1만,,💖디오디너리X해쭈 이벤트💖\n\n📣앙큼불여우들 주-목!!!\n영상 시청 후 댓글과 ...
2,https://youtu.be/Wr-mT74Gn2A&pp,공부,[공시생 브이로그2 #8] 경각심 좀 가져야 할 싹퉁바가지 컴백ㅣ살다 살다 별 초파...,뎌니Dyeoni,UCsL3gn0_x1BLzZAVmDwK8Hw,2023. 9. 18.,631,2.52천,유료 광고 포함,"인스타 알못,,,이지만\n최대한 피드 많이 올리려고 노력했는데\n내 마음 알아조,,..."
3,https://youtu.be/L8mMwsnZwkU&pp,공부,직장인 삼킨 최우식 & 멈추지 않는 알람⏰ 소리 #highlight #유료광고포함 ...,tvN,UC78PMQprrZTbU0IlMDsYZPw,2023. 4. 15.,"276,297",391만,유료 광고 포함,티빙에서 스트리밍 : https://tving.onelink.me/xHqC/30a7...
4,https://youtu.be/uBmj8nymexc&pp,공부,ENG) vlog 가을맞이 자취방 꾸미고 매운어묵김밥과 신라면 더레드 먹으며 매운 ...,Orosina 오로시나,UCgbu1qi6N17djEuulIrl20w,2023. 9. 17.,"21,232",15.6만,,#자취브이로그 #요리브이로그 #자취요리 \n\n⭐ https://www.instag...
...,...,...,...,...,...,...,...,...,...,...
655,https://youtu.be/eQN5EreRC9I,코디,애매한 날씨에 뭐 입지?💧 일교차 심한 날 봄옷 코디! #shorts,한별Hanbyul,UC9gW47NqzI1x7e8qsflvUUw,2023. 4. 26.,"187,897",74만,,💌 𝐂𝐨𝐧𝐭𝐚𝐜𝐭 fromhanbyul@gmail.com\n🥀 𝐢𝐧𝐬𝐭𝐚𝐠𝐚𝐦 @r...
656,https://youtu.be/ucOO0qsFS5A&pp,코디,무조건 개이득인 가성비 끝판왕 꿀템 4가지🔥,핏더사이즈,UCKWv0ScT6PpZcpg7ivUVWLA,2023. 9. 17.,"66,612",73.4만,,#가을코디 #데님셔츠 #팬츠추천 #니트추천\n\n브랜드 URL\n\n아노블리어\nh...
657,https://youtu.be/LWUIMv1L6mI,코디,대학생 필수 스니커즈 추천 BEST 3 유료광고 포함,추천모아,UCMBFotsUgoPKkzh4Ek5aESA,2023. 7. 23.,250,12,,#대학생패션 #대학생코디 #신발추천 #신발 #스니커즈추천 #스니커즈 #반스 #컨버스...
658,https://youtu.be/PbnY8ueDG5Q&pp,코디,OOTD VLOG! 빈티지 쇼핑하고 식물도 사고 비건 지향 식사와 운동을 하였습니다...,"Susan,",UC9edwufjEirbl9-2vns43VQ,2022. 9. 26.,"63,467",20.9만,유료 광고 포함,#유료광고 #아워비건 #비건지향 #ootd \n* 본 영상은 마녀공장(아워 비건)의...


In [24]:
# 구독자 수 계산
crawling_2_copy = crawling_2

for i in range(len(crawling_2_copy)):
    
    # 1,000
    if '천' in str(crawling_2_copy['구독자 수'][i]): 
        k_num = int(float(crawling_2_copy['구독자 수'][i].replace('천',''))*1000)
        crawling_2_copy.loc[i, '구독자 수'] = format(k_num)
        
    # 10,000
    elif '만' in str(crawling_2_copy['구독자 수'][i]): 
        m_num = int(float(crawling_2_copy['구독자 수'][i].replace('만',''))*10000)
        crawling_2_copy.loc[i, '구독자 수'] = format(m_num)

crawling_2_copy

,비디오 URL,키워드,제목,채널,채널ID,업로드 일자,조회수,구독자 수,유료광고 배너,더보기란
0,https://youtu.be/XlP8zzp0VH4&pp,공부,미리 정해두는 5월 필수 공부 (책 증정이벤트 & 유료광고 포함) (중고등 과목별 ...,대치동장원장,UCiNbRpsxCHAWNvqwJ5VEuGg,2023. 4. 8.,"7,387",33500,유료 광고 포함,대치동장원장의 자기주도학습 노하우를 체계적으로 마스터할 수 있는\nCLASSU 자기...
1,https://youtu.be/ejj-BAA7lwo&pp,공부,[유료광고포함] 새벽 4시 기상해서 미국 하이틴으로 변신하기! 미드 유포리아 캐씨 ...,해쭈 [HAEJOO],UC6KwCU8Y8Uw4h_Q0ptLZkqw,2023. 2. 10.,"1,116,226",751000,,💖디오디너리X해쭈 이벤트💖\n\n📣앙큼불여우들 주-목!!!\n영상 시청 후 댓글과 ...
2,https://youtu.be/Wr-mT74Gn2A&pp,공부,[공시생 브이로그2 #8] 경각심 좀 가져야 할 싹퉁바가지 컴백ㅣ살다 살다 별 초파...,뎌니Dyeoni,UCsL3gn0_x1BLzZAVmDwK8Hw,2023. 9. 18.,631,2520,유료 광고 포함,"인스타 알못,,,이지만\n최대한 피드 많이 올리려고 노력했는데\n내 마음 알아조,,..."
3,https://youtu.be/L8mMwsnZwkU&pp,공부,직장인 삼킨 최우식 & 멈추지 않는 알람⏰ 소리 #highlight #유료광고포함 ...,tvN,UC78PMQprrZTbU0IlMDsYZPw,2023. 4. 15.,"276,297",3910000,유료 광고 포함,티빙에서 스트리밍 : https://tving.onelink.me/xHqC/30a7...
4,https://youtu.be/uBmj8nymexc&pp,공부,ENG) vlog 가을맞이 자취방 꾸미고 매운어묵김밥과 신라면 더레드 먹으며 매운 ...,Orosina 오로시나,UCgbu1qi6N17djEuulIrl20w,2023. 9. 17.,"21,232",156000,,#자취브이로그 #요리브이로그 #자취요리 \n\n⭐ https://www.instag...
...,...,...,...,...,...,...,...,...,...,...
655,https://youtu.be/eQN5EreRC9I,코디,애매한 날씨에 뭐 입지?💧 일교차 심한 날 봄옷 코디! #shorts,한별Hanbyul,UC9gW47NqzI1x7e8qsflvUUw,2023. 4. 26.,"187,897",740000,,💌 𝐂𝐨𝐧𝐭𝐚𝐜𝐭 fromhanbyul@gmail.com\n🥀 𝐢𝐧𝐬𝐭𝐚𝐠𝐚𝐦 @r...
656,https://youtu.be/ucOO0qsFS5A&pp,코디,무조건 개이득인 가성비 끝판왕 꿀템 4가지🔥,핏더사이즈,UCKWv0ScT6PpZcpg7ivUVWLA,2023. 9. 17.,"66,612",734000,,#가을코디 #데님셔츠 #팬츠추천 #니트추천\n\n브랜드 URL\n\n아노블리어\nh...
657,https://youtu.be/LWUIMv1L6mI,코디,대학생 필수 스니커즈 추천 BEST 3 유료광고 포함,추천모아,UCMBFotsUgoPKkzh4Ek5aESA,2023. 7. 23.,250,12,,#대학생패션 #대학생코디 #신발추천 #신발 #스니커즈추천 #스니커즈 #반스 #컨버스...
658,https://youtu.be/PbnY8ueDG5Q&pp,코디,OOTD VLOG! 빈티지 쇼핑하고 식물도 사고 비건 지향 식사와 운동을 하였습니다...,"Susan,",UC9edwufjEirbl9-2vns43VQ,2022. 9. 26.,"63,467",209000,유료 광고 포함,#유료광고 #아워비건 #비건지향 #ootd \n* 본 영상은 마녀공장(아워 비건)의...


In [27]:
# 유료광고 배너 형태 확인
crawling_2_copy['유료광고 배너'].value_counts()

유료광고 배너
            404
유료 광고 포함    256
Name: count, dtype: int64

In [ ]:
crawling_2_copy['업로드 일자'] = crawling_2_copy['업로드 일자'].str.replace('(\d{4}.\d{2}.\d{2}).')

In [35]:
# 유료광고 배너 & 더보기란 필터링
filtered_df = crawling_2_copy[(crawling_2_copy['유료광고 배너'] == '유료 광고 포함') & (~crawling_2_copy['더보기란'].str.contains('광고를 포함하지 않습니다|광고를 포함하고 있지 않습니다'))]

# 업로드 일자 날짜만 추출
filtered_df['업로드 일자'] = filtered_df['업로드 일자'].str.replace('최초 공개: ', '')

filtered_df

C:\Users\SANDBOX\AppData\Local\Temp\ipykernel_7184\3220311295.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['업로드 일자'] = filtered_df['업로드 일자'].str.replace('최초 공개: ', '')


,비디오 URL,키워드,제목,채널,채널ID,업로드 일자,조회수,구독자 수,유료광고 배너,더보기란
0,https://youtu.be/XlP8zzp0VH4&pp,공부,미리 정해두는 5월 필수 공부 (책 증정이벤트 & 유료광고 포함) (중고등 과목별 ...,대치동장원장,UCiNbRpsxCHAWNvqwJ5VEuGg,2023. 4. 8.,"7,387",33500,유료 광고 포함,대치동장원장의 자기주도학습 노하우를 체계적으로 마스터할 수 있는\nCLASSU 자기...
2,https://youtu.be/Wr-mT74Gn2A&pp,공부,[공시생 브이로그2 #8] 경각심 좀 가져야 할 싹퉁바가지 컴백ㅣ살다 살다 별 초파...,뎌니Dyeoni,UCsL3gn0_x1BLzZAVmDwK8Hw,2023. 9. 18.,631,2520,유료 광고 포함,"인스타 알못,,,이지만\n최대한 피드 많이 올리려고 노력했는데\n내 마음 알아조,,..."
3,https://youtu.be/L8mMwsnZwkU&pp,공부,직장인 삼킨 최우식 & 멈추지 않는 알람⏰ 소리 #highlight #유료광고포함 ...,tvN,UC78PMQprrZTbU0IlMDsYZPw,2023. 4. 15.,"276,297",3910000,유료 광고 포함,티빙에서 스트리밍 : https://tving.onelink.me/xHqC/30a7...
6,https://youtu.be/qabCoid-K-8&pp,공부,𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭 부장님도 같이 리듬 타게 되는 직장인 노동요,때껄룩ᴛᴀᴋᴇ ᴀ ʟᴏᴏᴋ,UCVut4hqvrjQC4qDE3oc5qig,2022. 10. 18.,"1,464,304",1490000,유료 광고 포함,"* 본 콘텐츠는 눈곰이 의 지원을 받아 함께 제작하였으며, \n유료 광고를 포함하고..."
7,https://youtu.be/iiXDssiA_Ts&pp,공부,문해력 이슈 | 문제는 영상 때문이다🤔? #highlight #유료광고포함#유퀴즈온...,유 퀴즈 온 더 튜브,UC920m3pMPH45qztdhppZhwA,2023. 7. 27.,"661,196",857000,유료 광고 포함,티빙에서 스트리밍 : https://tving.onelink.me/xHqC/30a7...
...,...,...,...,...,...,...,...,...,...,...
648,https://youtu.be/IwUaaEd_eYA&pp,코디,발목뚠뚠❌ 얇고쫀득해 다리가예뻐보이는 만능코디템 🧦기본&니트양말🧦 ㅣ 전직 다리모델...,뷰티트레이너 쭈언니,UCGr3FB8OwSwNwk2FEpCGxPQ,2023. 9. 17.,"17,032",115000,유료 광고 포함,*더 많은 코디는 인스타그램 하이라이트에 https://www.instagram.c...
650,https://youtu.be/3BA3k748akY&pp,코디,인⭐그램 업로드에 빠진 이효리 남다른(?) 출근길 스타일링 #유료광고포함 #댄스가수...,tvN,UC78PMQprrZTbU0IlMDsYZPw,2023. 6. 16.,"62,479",3910000,유료 광고 포함,티빙에서 스트리밍 : https://tving.onelink.me/xHqC/30a7...
653,https://youtu.be/-G5ieAhGAqY,코디,인스타 반응이 너무 폭발적이어서 안 올릴 수가 없었던 이 날 코디🫢 #윗유하우스,나나씨 Nanasi,UCzBbE7x7mPrs56PvCc0errA,2022. 10. 9.,"598,977",98800,유료 광고 포함,"instagram, tiktok @nana6a6y ◡̈\n\n이 영상은 '유료광고'..."
658,https://youtu.be/PbnY8ueDG5Q&pp,코디,OOTD VLOG! 빈티지 쇼핑하고 식물도 사고 비건 지향 식사와 운동을 하였습니다...,"Susan,",UC9edwufjEirbl9-2vns43VQ,2022. 9. 26.,"63,467",209000,유료 광고 포함,#유료광고 #아워비건 #비건지향 #ootd \n* 본 영상은 마녀공장(아워 비건)의...


In [46]:
# filtered_df.to_csv('모니터링 명단_크롤링 시도.csv', encoding='utf-8-sig')

In [36]:
# 결과 구글 시트에 저장
gc = pygsheets.authorize(service_account_file='creds.json')
sheetname= '모니터링 명단_크롤링 시도'
sh = gc.open(sheetname)

# 오류 발생 시, worksheet의 행 수 늘려서 재실행
wks = sh.worksheet_by_title('광고 시장')
wks.set_dataframe(filtered_df, 'A1', index=False)